### Import pakietów

In [8]:
import os
import json
import requests
from bs4 import BeautifulSoup

### Narzędzia

In [5]:
def extract(ancestor, selector=None, attribute=None, multiple=False):
    if selector:
        if multiple:
            if attribute:
                return [tag(attribute).strip() for tag in ancestor.select(selector)]
            return [tag.text.strip() for tag in ancestor.select(selector)]
    
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute].strip()
    return ancestor.text.strip()


In [6]:
selectors = {
                    "opinion_id": (None, "data-entry-id"),
                    "author": ("span.user-post__author-name",),
                    "recommendation": ("span.user-post__author-recomendation > em",),
                    "stars": ("span.user-post__score-count",),
                    "content": ("div.user-post__text",),
                    "pros": ("div.review-feature__item--positive", None, True),
                    "cons": ("div.review-feature__item--negative", None, True),
                    "useful": ("button.vote-yes > span",),
                    "unuseful": ("button.vote-no > span",),
                    "post_date": ("span.user-post__published > time:nth-child(1)",'datetime'),
                    "purchase_date": ("span.user-post__published > time:nth-child(2)",'datetime'),
                }

### Proces ekstrakcji opinii o produkcie ze strony Ceneo.pl

In [9]:
with open("./headers.json", "r") as jf:
    headers = json.load(jf)

In [17]:

product_id = "65035604"
next_page = f"https://www.ceneo.pl/{product_id}#tab=reviews"
all_opinions = []
while next_page:
    print(next_page)
    response = requests.get(next_page, headers=headers)
    if response.status_code == 200:
        page_dom = BeautifulSoup(response.text, "html.parser")
        opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")
        for opinion in opinions:
            single_opinion = {
                key: extract(opinion, *value)
                for key, value in selectors.items()

            }
            all_opinions.append(single_opinion)
        try:
            next_page = "https://www.ceneo.pl"+extract(page_dom, "a.pagination__next", "href")
        except TypeError:
            next_page = None
    else: print(response.status_code)



https://www.ceneo.pl/65035604#tab=reviews
https://www.ceneo.pl/65035604/opinie-2
https://www.ceneo.pl/65035604/opinie-3
https://www.ceneo.pl/65035604/opinie-4
https://www.ceneo.pl/65035604/opinie-5
https://www.ceneo.pl/65035604/opinie-6
https://www.ceneo.pl/65035604/opinie-7
https://www.ceneo.pl/65035604/opinie-8
https://www.ceneo.pl/65035604/opinie-9
https://www.ceneo.pl/65035604/opinie-10
https://www.ceneo.pl/65035604/opinie-11
https://www.ceneo.pl/65035604/opinie-12
https://www.ceneo.pl/65035604/opinie-13
https://www.ceneo.pl/65035604/opinie-14
https://www.ceneo.pl/65035604/opinie-15
https://www.ceneo.pl/65035604/opinie-16
https://www.ceneo.pl/65035604/opinie-17
https://www.ceneo.pl/65035604/opinie-18
https://www.ceneo.pl/65035604/opinie-19
https://www.ceneo.pl/65035604/opinie-20
https://www.ceneo.pl/65035604/opinie-21
https://www.ceneo.pl/65035604/opinie-22
https://www.ceneo.pl/65035604/opinie-23
https://www.ceneo.pl/65035604/opinie-24
https://www.ceneo.pl/65035604/opinie-25
https:

### Zapisanie wyników do bazy danych

In [18]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")

In [19]:
with open(f"./opinions/{product_id}.json", "w", encoding="UTF-8") as jf:
    json.dump(all_opinions, jf, indent=4, ensure_ascii=False)